<a href="https://colab.research.google.com/github/bigluck07/Code_States_Project/blob/main/%EC%84%B9%EC%85%982_Proj_DS_HR_%EC%9D%B4%EB%8C%80%EC%9A%B4(%EB%B0%9C%ED%91%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터

해당 데이터는 kaggle의  **R Analytics: Job Change of Data Scientists**의 데이터 셋입니다.

https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists


### Note :
- 데이터 세트가 불균형합니다.
- 대부분의 기능은 범주 형 (명목, 순서, 이진)이며 일부는 카디널리티가 높습니다.
- 누락 된 대치도 파이프 라인의 일부가 될 수 있습니다.


### 특성설명:
- enrollee_id : 후보자의 고유 ID
- city : 도시 코드
- city_ development _index : 도시 개발 지수 (축척)
- gender : 후보자의 성별
- relevent_experience : 후보자의 관련 경험
- enrolled_university : 등록 된 대학 과정 유형 (있는 경우)
- education_level : 지원자의 교육 수준
- major_discipline : 후보자 교육 전공
- experience : 근무기간(년)
- company_size : 현재 고용주 회사의 직원 수
- company_type : 현재 고용주의 유형
- lastnewjob : 이전 직장과 현재 직장 간의 연도 차이
- training_hours : 완료된 교육 시간
- target : 0 – 전직을 찾고 있지 않음, 1 – 전직을 찾고 있음

## **1) 데이터 선정 이유 및 문제 정의**

### 시나리오
빅데이터와 데이터 사이언스에 활동하는 'Data State'는 회사가 진행하는 일부 과정을 성공적으로 통과한 사람들 중에서 **데이터 과학자를 고용**하기를 원합니다. 

많은 사람들이 회사의 교육에 신청하므로 데이터의 양에는 문제가 없습니다. 

기업은 이들 후보자 중 누가 훈련이나 신규 취업을 원하는지 알고 싶어하는데, 이는 교육의 질이나 과정 계획, 후보자의 분류뿐 아니라 비용과 시간을 줄이는 데 도움이 되기 때문입니다. 

인구통계학, 교육, 경험과 관련된 정보는 후보자 등록과 정규 수강생 등록에서 수집하였습니다.


이 데이터 세트는 인사 연구를 위해 현재 직장을 그만두게 하는 요인을 이해하기 위해 고안되었습니다. 

현재 자격 증명, 인구 통계, 경험 데이터를 사용하는 모델에 의해 당신은 후보자가 새로운 일자리를 찾거나 회사에서 일할 확률을 예측할 뿐만 아니라 직원 결정에 영향을 미치는 요인을 해석하게 됩니다.

학습 및 테스트를 위해 분할 된 전체 데이터이며, 샘플 제출은 enrolee _id, target 열과 함께 제공된 테스트 세트의 enrollee_id에 해당합니다.

# 분석

## **2) 데이터를 이용한 가설 및 평가지표, 베이스라인 선택**
가설:
1. 도시개발점수가 낮은 곳에 사는 교육생은 도시개발점수가 높은곳으로 이직하고 싶어할 것이다.
2. 근속 3년 이상인 교육생은 이직하려고 할 것이다.
3. training_hours가 높은 교육생은 이직하고자 할 것이다.

### **태스크를 수행한 후, 다음 질문에 대답할 수 있어야 합니다.**

1. target = 이직을 했는지, 안했는지
 - 교육생의 데이터를 통해 이직하고자 하는 교육생을 판별하고, 이직을 권유하기 위해서
2. 해당 베이스라인 모델과 평가지표를 선택한 이유를 설명하세요.
 - 베이스라인: train데이터의 target변수의 value_ciunts 중 결과치가 높은 value로 모든 결과를 예측한다고 했을때 갖게되는 모델로, 예측을 하지않고 target변수가 가지고 있는 요소 중 함유량이 높은 요소로 모든 예측값을 결정한 기본이 되는 모델이다. 따라서 해당 베이스라인은 '모든 교육생이 이직을 원하지 않는다'를 예측하고 틀린부분들은 이직을 할 교육생이라는 의미를 가지고 있다.

 - 평가지표 
    - 'accuracy': 모델의 정확도를 확인해주는 평가지표로 예측에 대한 맞은 정도를 알려준다. 따라서 기준모델에서는 2개의 요소 중 가장많은 요소의 비율이 보여진다고 할 수 있다.

    - 'AUC': 'accuracy'의 단점이될 수 있는 모델이 한 요소로 모두 찍었을 경우 보일 수 있는 높은 정확도를 구분할 수 있는 지표로 사용하면서도, 고정수치이기때문에 얼마나 모델이 잘 예측하는지 절대적 수치로 측정할 수 있으며, 분류 임계치가 고정되어있기에, 모델의 분류 임계점이 어떠한지에 관계없이 모델이 잘 예측하는지 측정할 수 있다.

In [ ]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap
    !pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# data read
data = pd.read_csv('/content/aug_train.csv')
data.shape

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=2)
train.shape, test.shape

#### 기준모델

In [ ]:
# 기준모델
target='target'
print(train[target].value_counts())

# 기준모델 정확도 계산
from sklearn.metrics import accuracy_score, roc_auc_score
major = train[target].mode()[0]
y_pred = [major] * len(train[target])
print("baseline accuracy: ", accuracy_score(train[target], y_pred))
print('AUC score: ', roc_auc_score(train[target], y_pred))

# baseline accuracy:  0.7512070990473705
# AUC score:  0.5

## **3) EDA와 데이터 전처리**

가설을 정했고 베이스라인을 만들었다면 데이터의 탐색 및 가공을 시작해봅니다. 바로 적용이 될 수 있는 데이터도 있겠지만, 대부분의 데이터는 전처리 작업이 필요합니다.
데이터 전처리를 통해 할 수 있는 것은 다양하지만 다음을 시도해볼 수 있습니다.

- EDA
- Feature Engineering
- 데이터의 정규화
- 노이즈 제거
- 결측치 제거 혹은 대체
- 데이터 밸런스
- 그 외

### **태스크를 수행한 후, 다음 질문에 대답할 수 있어야 합니다**

1. Data Leakage가 있었나요? 없었다면 어떻게 방지했나요?
- 분류모델이라 target 피쳐를 사용하여 FE를 하거나, 연관된 피쳐가 없다.

2. 만들고자 하는 모델은 언제 유용한가요? 어떤 한계를 가지고 있나요?
-

In [ ]:
print(train.shape, '\n')
train.head()

In [ ]:
# 칼럼확인
train.columns

In [ ]:
# 형식확인
train.dtypes

In [ ]:
# describe
train.describe(include='all').T

In [ ]:
# 결측치 확인
[(x, train[x].isnull().sum()) for x in train.columns if train[x].isnull().any()]

In [ ]:
from pandas_profiling import ProfileReport
#profile = ProfileReport(train, minimal=True).to_notebook_iframe()

# Warnings

# 'city' has a high cardinality: 123 distinct values	High cardinality
# 'enrollee_id' unique values

#### EDA

In [ ]:
# EDA
def removecity(i):
  i = i.replace('city_','')
  i = int(i)
  return i

def clear(df):
  # target 형변환
  df['target'] = df['target'].apply(int)

  # city의 인코딩을 순서적으로 만들기
  df['city'] = df['city'].apply(removecity)

  # city_development_index 소수점 셋째자리 반올림
  df.loc[:, "city_development_index"] = round(df.loc[:, "city_development_index"],3)

  # 10% 이하 결측치 제거
  df.dropna(subset=["experience"], inplace=True)
  df.dropna(subset=["enrolled_university"], inplace=True)
  df.dropna(subset=["last_new_job"], inplace=True)
  df.dropna(subset=["education_level"], inplace=True)

  # 10% 초과 결측치는 imputer 활용

    # 무의미한 피쳐 삭제
  need_del = ['enrollee_id']
  df = df.drop(need_del, axis=1)
  
  return df

In [ ]:
train_cl = clear(train)
test_cl = clear(test)

In [ ]:
train_cl['city_development_index'].unique()

In [ ]:
train_cl.shape, test_cl.shape

In [ ]:
feature = train.columns.drop('target')

#### FE

In [ ]:
# Feature Engineering - 건질게 없느냐

# # city_development_index, city
# city_dvp = pd.DataFrame(df.groupby('city_development_index')['city'].value_counts())
# train['city_development_index'].unique()

# company_size company_type
# company = df['company_type'] + ',' + df['company_size'] - 하나가 NaN이면 NaN으로 표시됨..;;

# del feature
# need_del = ['company_type','company_size']
# df = df.drop(need_del, axis=1)


In [ ]:
# train_cl.loc[:, ["company_size","company_type"]].isna()#.value_counts()# and train_cl.loc[:, "company_type"].isna()]
# train_cl[train_cl.loc[:, "company_size"].isna() == train_cl.loc[:, "company_type"].isna()].isna().sum()
# train_cl[train_cl.loc[:, "company_size"].isna() != train_cl.loc[:, "company_type"].isna()]['company_size'].value_counts()

## **4) 머신러닝 방식 적용 및 교차검증**

데이터의 탐색과 전처리 작업이 끝났다면 모델링을 통해 베이스라인과의 성능 비교를 해봅니다.

- Linear / Tree-based / Ensemble / Boosting 모델을 학습하세요. (다양하게 시도해보시는 걸 추천합니다.)
- 평가지표를 계산 후 베이스라인과 비교해보세요.
- 어느정도 성능이 나왔다면, 교차 검증 (이하 CV)을 통해서 일반화될 가능성이 있는지 확인해봅니다.
- 모델 성능을 개선하기 위한 다양한 방법을 적용해보세요.
    - Hyperparameter tuning, etc.
- 최소 2개 이상의 모델을 만들어서 validation 점수를 보고하세요.
- 최종 모델의 test 점수를 보고하세요.

### **태스크를 수행한 후, 다음 질문에 대답할 수 있어야 합니다.**

1. 모델을 학습한 후에 베이스라인보다 잘 나왔나요? 그렇지 않다면 그 이유는 무엇일까요?
 - 잘나왔습니다. 아마 결측치가 별로없는 데이터는 삭제하고, 평균값으로 나머지를 채워서 그런거 같습니다.
2. 모델 성능 개선을 위해 어떤 방법을 적용했나요? 그 방법을 선택한 이유는 무엇인가요?
 - 랜더마이즈cv를 통해 최적의 하이퍼파라미터값을 찾았습니다. 파라미터를 개별적으로 손대는것은 너무나 많은 시간이 들기때문에, 임의적으로돌린 후 최적의 하이퍼 파라미터를 찾았습니다.
3. 최종 모델에 관해 설명하세요.
 - LBGM에 랜더마이즈cv를 사용후 Boosting을 활용한 모델로 랜더마이즈cv로 최적의 하이퍼파라미터를 찾고, 해당 값을 통해 부스팅을 한 모델입니다.

In [ ]:
# data split
from sklearn.model_selection import train_test_split
train_f, val_f = train_test_split(train_cl, test_size=0.2, random_state=2)


target = 'target'
feature = train_f.columns.drop(target)

X_train = train_f[feature]
y_train = train_f[target]
X_val = val_f[feature]
y_val = val_f[target]
X_test = test_cl[feature]
y_test = test_cl[target]

In [ ]:
# action_type: ordi
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier, plot_importance

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# 기준모델
target='target'
train[target].value_counts()

# 기준모델 정확도 계산
from sklearn.metrics import accuracy_score
major = train[target].mode()[0]
y_pred = [major] * len(train[target])
print("baseline accuracy: ", accuracy_score(train[target], y_pred))

####  Tree-based

In [ ]:
# make pipe
Tree_based_pipe = make_pipeline(
    TargetEncoder(smoothing=10.0), 
    SimpleImputer(strategy='median'), 
    StandardScaler(),
    DecisionTreeClassifier(random_state=2, max_depth=4)
)

Tree_based_pipe.fit(X_train,y_train)

print('모델 정확도', Tree_based_pipe.score(X_train, y_train))
print('검증 정확도', Tree_based_pipe.score(X_val, y_val))

y_pred_proba = Tree_based_pipe.predict_proba(X_val)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_val, y_pred_proba))


from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')

##### best parameter

In [ ]:
# find best parameter - RandomizedSearchCV
from scipy.stats import randint, uniform


Tree_based_pipe = make_pipeline(
    TargetEncoder(), 
    SimpleImputer(), 
    StandardScaler(), 
    DecisionTreeClassifier(random_state=2)
)


dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'decisiontreeclassifier__max_depth' : [2,4,6,8,10,12,None]
}

Tree_based_clf = RandomizedSearchCV(
    Tree_based_pipe, 
    param_distributions=dists, 
    n_iter=10, 
    cv=3,
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

Tree_based_clf.fit(X_train, y_train);
Tree_based_pipe_rd = Tree_based_clf.best_estimator_

In [ ]:
print('최적 하이퍼파라미터: ', Tree_based_clf.best_params_)
print('MAE: ', -Tree_based_clf.best_score_)

In [ ]:
print('모델 정확도', Tree_based_pipe_rd .score(X_train, y_train))
print('검증 정확도', Tree_based_pipe_rd .score(X_val, y_val))

y_pred_proba = Tree_based_pipe_rd .predict_proba(X_val)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_val, y_pred_proba))


from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')

#### Ensemble

In [ ]:
# make pipe
Ensemble_pipe = make_pipeline(
    TargetEncoder(smoothing=10.0), 
    SimpleImputer(strategy='median'), 
    StandardScaler(), 
    RandomForestClassifier(random_state=2, max_depth=7, n_jobs=-1, n_estimators=50)
)

Ensemble_pipe.fit(X_train,y_train)
print('모델 정확도', Ensemble_pipe.score(X_train, y_train))
print('검증 정확도', Ensemble_pipe.score(X_val, y_val))

y_pred_proba = Ensemble_pipe.predict_proba(X_val)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_val, y_pred_proba))
print('\n')


fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')

##### best parameter

In [ ]:
# find best parameter - RandomizedSearchCV
from scipy.stats import randint, uniform

Ensemble_pipe = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    StandardScaler(), 
    RandomForestClassifier(random_state=2)
)

dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 1000), 
    'randomforestclassifier__max_depth': [2,4,6,8,10,12,None],
    'randomforestclassifier__max_features': uniform(0, 1) 
}

Ensemble_clf = RandomizedSearchCV(
    Ensemble_pipe, 
    param_distributions=dists, 
    n_iter=10, 
    cv=3,
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

Ensemble_clf.fit(X_train, y_train);
Ensemble_pipe_rd = Ensemble_clf.best_estimator_

In [ ]:
print('최적 하이퍼파라미터: ', Ensemble_clf.best_params_)
print('MAE: ', -Ensemble_clf.best_score_)

In [ ]:
print('모델 정확도', Ensemble_pipe_rd .score(X_train, y_train))
print('검증 정확도', Ensemble_pipe_rd .score(X_val, y_val))

y_pred_proba = Ensemble_pipe_rd .predict_proba(X_val)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_val, y_pred_proba))


from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')

#### Boosting

In [ ]:
from category_encoders import OrdinalEncoder

encoder = TargetEncoder(smoothing=10.0)
X_train_encoded = encoder.fit_transform(X_train, y_train) # 학습데이터
X_val_encoded = encoder.transform(X_val) # 검증데이터

imputer = SimpleImputer(strategy='median')
X_train_im = imputer.fit_transform(X_train_encoded)
X_val_im = imputer.transform(X_val_encoded)

boosting = LGBMClassifier(random_state=2, 
                          max_depth=3, 
                          n_jobs=-1, 
                          n_estimators=1000, 
                          num_leaves=31,
                          learning_rate=0.2)

eval_set = [(X_train_im, y_train), 
            (X_val_im, y_val)]

boosting.fit(X_train_im, y_train, 
          eval_set=eval_set,
          early_stopping_rounds=50
         )
# training's binary_logloss: 0.421149	
# valid_1's binary_logloss: 0.423476

In [ ]:
s = X_train['city'].unique()
s

In [ ]:
X_train_encoded['city'].unique()

In [ ]:

y_pred = boosting.predict(X_val_im)
print('모델 정확도', boosting.score(X_train_im, y_train))
print('검증 정확도', boosting.score(X_val_im, y_val))

y_pred_proba = boosting.predict_proba(X_val_im)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_val, y_pred_proba))
print('\n')

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')

##### Best parameter

In [ ]:
# find best parameter - RandomizedSearchCV
from scipy.stats import randint, uniform

Boosting_pipe = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    StandardScaler(), 
    LGBMClassifier(random_state=2)
)

dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'lgbmclassifier__n_estimators': randint(50, 1000), 
    'lgbmclassifier__max_features': uniform(0, 1),
    'lgbmclassifier__num_leaves':[10,20,30,40],
    'lgbmclassifier__learning_rate': [0.1,0.2,0.3,0.4],
    'lgbmclassifier__max_depth': [2,4,6,8,10,12,None]
}

boosting_clf = RandomizedSearchCV(
    Boosting_pipe, 
    param_distributions=dists, 
    n_iter=10, 
    cv=3,
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

boosting_clf.fit(X_train, y_train);
Boosting_pipe_rd = boosting_clf.best_estimator_

In [ ]:
print('최적 하이퍼파라미터: ', boosting_clf.best_params_)
print('MAE: ', -boosting_clf.best_score_)

In [ ]:
encoder = TargetEncoder(smoothing=500.0, min_samples_leaf=8)
X_train_encoded = encoder.fit_transform(X_train, y_train) # 학습데이터
X_val_encoded = encoder.transform(X_val) # 검증데이터
X_test_encoded = encoder.transform(X_test) # 테스트데이터

imputer = SimpleImputer(strategy='median')
X_train_im = imputer.fit_transform(X_train_encoded)
X_val_im = imputer.transform(X_val_encoded)
X_test_im = imputer.transform(X_test_encoded)

boosting_rd = LGBMClassifier(random_state=2, 
                          max_depth=None, 
                          n_jobs=-1, 
                          n_estimators=102, 
                          num_leaves=20,
                          learning_rate=0.1,
                          max_features = 0.665)

eval_set = [(X_train_im, y_train), 
            (X_val_im, y_val)]

boosting_rd.fit(X_train_im, y_train, 
          eval_set=eval_set,
          early_stopping_rounds=50
         )

In [ ]:
y_pred = boosting_rd.predict(X_val_im)
print('모델 정확도', boosting_rd.score(X_train_im, y_train))
print('검증 정확도', boosting_rd.score(X_val_im, y_val))

y_pred_proba = boosting_rd.predict_proba(X_val_im)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_val, y_pred_proba))
print('\n')

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')


#### 모델 비교
##### tree-based 모델
- 기본
  - 검증 정확도 0.7930675909878683
  - AUC score:  0.7933806367649752

- 랜더마이즈cv
  - 검증 정확도 0.7937608318890814
  - AUC score:  0.7958575078287049

##### Ensemble 모델
- 기본
  - 검증 정확도 0.8031195840554592
  - AUC score:  0.8128968741029947

- 랜더마이즈cv
  - 검증 정확도 0.8055459272097054
  - AUC score:  0.8129621403698811

##### Boosting 모델
- 기본
  - 검증 정확도 0.8010398613518197
  - AUC score:  0.8148762928240876

- 랜더마이즈cv_부스팅
  - 검증 정확도 0.8010398613518197
  - AUC score:  0.8161819428696117


##### 가장높은 AUC 모델: Boosting모델의 랜더마이즈cv+부스팅

#### 최종모델

In [ ]:
y_pred = boosting_rd.predict(X_test_im)

y_pred_proba = boosting_rd.predict_proba(X_test_im)[:, -1]
print('\n')
print('AUC score: ', roc_auc_score(y_test, y_pred_proba))
print('\n')

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.scatter(fpr, tpr, color='blue')
plt.plot(fpr, tpr, color='green')
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')


## **5) 머신러닝 모델 해석**

프로젝트에서 가장 중요하다고 볼 수 있는 부분 입니다. 우리는 SHAP, PDP 등을 통해서 모델이 관측치를 어떤 특성을 활용했거나, 어떤 특성이 타겟에 영향을 끼쳤는지 등을 해석하는 방법에 대해서 배웠습니다.
여러분의 프로젝트에도 이러한 해석 방법을 활용해 머신러닝 모델을 비전문가라도 조금 더 쉽게 이해하고 접근할 수 있도록 해주셔야 합니다.

- PDP, SHAP을 활용하여 최종 모델을 설명합니다
- 시각화는 "설명"이 제일 중요합니다.

### **태스크를 수행한 후, 다음 질문에 대답할 수 있어야 합니다.**

1. 모델이 관측치를 예측하기 위해서 어떤 특성을 활용했나요?
2. 어떤 특성이 있다면 모델의 예측에 도움이 될까요? 해당 특성은 어떻게 구할 수 있을까요?

In [ ]:
# dpi(dots per inch) 수치를 조정해 이미지 화질 조정
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 144

# 그래프 그리기
from pdpbox.pdp import pdp_isolate, pdp_plot

model = boosting_rd

# 그래프를 그려야하는 특성
features = X_train_encoded.columns
# 그래프를 그리는 함수
def pdp_plt(feature):
  isolated = pdp_isolate(
      model=model, # 모델
      dataset=X_val_encoded, #검증데이터
      model_features=X_train_encoded.columns, 
      feature=feature, # 확인하고자 하는 특성
      grid_type='percentile', # default='percentile', or 'equal' # percentile: 데이터의 분포대로 해줌, equal: 점을 균일하게 찍어줌
      num_grid_points=100 # default=10
  )
  return pdp_plot(isolated, feature_name=feature);

In [ ]:
features = X_train_encoded.columns
for i in features:
  pdp_plt(i)

### 가설확인

#### 가설1
1. 도시개발점수가 낮은 곳에 사는 교육생은 도시개발점수가 높은곳으로 이직하고 싶어할 것이다.
  - pdp 'city_development_index' 확인
   > 0.6이하에서 이직이 빈번하게 일어나며, 0.92인 도시들는 이직율이 높다


In [ ]:
# 양의 관계 - city, city_development_index, gender, relevent_experience, enrolled_university, education_level, major_disciline, experience, company_size, company_type
pdp_plt('city_development_index')

In [ ]:
# 2 features
from pdpbox.pdp import pdp_interact, pdp_interact_plot
features = ['city', 'city_development_index']

# 두 특성상의 의존성을 확인
interaction = pdp_interact(
    model=boosting_rd, 
    dataset=X_train_encoded,
    model_features=X_train_encoded.columns, 
    features=features
)

pdp_interact_plot(interaction, plot_type='grid', 
                  feature_names=features);

#### 가설2
2. 근속 3년 이상인 교육생은 이직하려고 할 것이다.
  > 4,2,3년차 순으로 이직률이 높으며 1년차에서도 일어난다.


In [ ]:
pdp_plt('last_new_job')

#### 가설3
3. training_hours가 높은 교육생은 이직하고자 할 것이다.
 > 150시간 이하의 교육 받은 교육생이 이직을 하는 경향이 크다.

In [ ]:
pdp_plt('training_hours')

### SHAP

In [ ]:
# 1. force plot - all
shap.initjs()

import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train_encoded[:100])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_train)

In [ ]:
shap_values = explainer.shap_values(X_train_encoded)
shap.summary_plot(shap_values[0], X_train_encoded, plot_type='bar')

### PDP

In [ ]:
# 양의 관계 - city, city_development_index, gender, relevent_experience, enrolled_university, education_level, major_disciline, experience, company_size, company_type
# 애매한괸계 - last_new_job, training_hours

In [ ]:
# permutation importance
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model,
    scoring='neg_mean_absolute_error', 
    n_iter=5,
    random_state=2
)

permuter.fit(X_train_encoded, y_train);

import pandas as pd
feature_names = X_train_encoded.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None,
    feature_names=feature_names
)

In [ ]:
# 2 features
from pdpbox.pdp import pdp_interact, pdp_interact_plot
features = ['last_new_job', 'training_hours']

# 두 특성상의 의존성을 확인
interaction = pdp_interact(
    model=boosting_rd, 
    dataset=X_train_encoded,
    model_features=X_train_encoded.columns, 
    features=features
)

pdp_interact_plot(interaction, plot_type='grid', 
                  feature_names=features);

In [ ]:
# 2 features
from pdpbox.pdp import pdp_interact, pdp_interact_plot
features = ['company_type', 'company_size']

# 두 특성상의 의존성을 확인
interaction = pdp_interact(
    model=boosting_rd, 
    dataset=X_train_encoded,
    model_features=X_train_encoded.columns, 
    features=features
)

pdp_interact_plot(interaction, plot_type='grid', 
                  feature_names=features);

In [ ]:
# 양의 관계 - city, city_development_index, gender, relevent_experience, enrolled_university, education_level, major_disciline, experience, company_size, company_type
# 애매한괸계 - last_new_job, training_hours

### 결론


- clty: 지역번호가 높을 수록 이직이 빈번하며, 70-105, 160이상 의 도시들이 이직이 할 확률이 더 높다.
- city_de_in: 0.6이하에서 이직이 빈번하게 일어나며, 0.9 전후의 몇 도시들안에서 이직이 일어난다.
- gender: ahems 모든성별에서 이직이 일어나지만, 0.27(female)과 0.25(other)가 확률적으로 높다
- relevent_experience: 둘다 이직을 하지만 관련경험이 없는경우 더 빈번하다
- enrolled_university: 풀타임코스에서 이직이 더욱 빈번하다
- education_level: 학력이 학사(0.27378286), 석사(0.20520194)인경우 빈번하다
- major_discipline: 전공이 존재하면 이직을 할 경우가 높지만, STEM(0.2544967), Business Degree(0.25504855)의 경우 더 빈번하다
- experience: 20년 이상의 경력이 있는 교육생 이외는 모두 이직할 가능성이 있지만, 1년미만과 1년, 3년의 경력을 가진 교육생이 가장 높다
- company_size: 10/49의 규모를 가진 회사가 이직률이 가장 높지만 유의미 한 수치는 아니다.
- company_type: 'Other','Early Stage Startup이 가장 높다
- last_new_job: 4,2,3년차 순으로 이직률이 높으며 1년차에서도 일어난다.
- training_hours: 150시간 이하의 교육시간을 갖은 교육생의 이직률이 높다